In [2]:
import pandas as pd

# Tamaño del chunk
chunk_size = 1000000  

# Inicializar un DataFrame vacío para almacenar el resultado
review_df = pd.DataFrame(columns=['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date'])

# Iterar sobre los chunks
for chunk in pd.read_json('./data/dataset_review.json', lines=True, chunksize=chunk_size):
    # Calcular el número de reseñas por negocio para el chunk actual
    # Eliminar filas con valores nulos
    review_df = pd.concat([review_df, chunk], ignore_index=True)

print(review_df.head())

    


/tmp/ipykernel_26450/3660312778.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  review_df = pd.concat([review_df, chunk], ignore_index=True)


                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

  stars useful funny cool                                               text  \
0     3      0     0    0  If you decide to eat here, just be aware it is...   
1     5      1     0    1  I've taken a lot of spin classes over the year...   
2     3      0     0    0  Family diner. Had the buffet. Eclectic assortm...   
3     5      1     0    1  Wow!  Yummy, different,  delicious.   Our favo...   
4     4      1     0    1  Cute interior and owner (?) gave us tour of up...   

                 date  
0 2018-07-07 22:09:11  
1 2012-0

In [3]:
display(review_df.head())

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [4]:
review_df.describe()

,date
count,6990280
mean,2017-01-11 11:22:33.441780992
min,2005-02-16 03:23:22
25%,2015-01-25 04:53:50.249999872
50%,2017-06-03 01:26:07
75%,2019-05-23 00:02:46.249999872
max,2022-01-19 19:48:45


In [5]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Convertir texto a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y puntuación
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenización
    tokens = word_tokenize(text)
    # Eliminar palabras vacías y lematización
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Unir tokens de nuevo en texto
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

# Aplicar preprocesamiento de texto a las reseñas
review_df['clean_text'] = review_df['text'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rodari78/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/rodari78/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/rodari78/nltk_data...


In [9]:
from textblob import TextBlob


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob

# Calcular el sentimiento de cada reseña
review_df['sentiment'] = review_df['clean_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Histograma de los sentimientos de las reseñas
plt.figure(figsize=(10, 6))
sns.histplot(data=review_df, x='sentiment', bins=20, kde=True)
plt.title('Distribución de sentimientos de las reseñas')
plt.xlabel('Sentimiento')
plt.ylabel('Frecuencia')
plt.show()


NameError: name 'review_df' is not defined